In [ ]:
# %matplotlib widget

# Needs this line or else pandas plots do not show in jupyterlab
%matplotlib inline

import rosbag_pandas
import pandas as pd

from matplotlib.pyplot import *

sys.path.append('../src')
from frame_conversions import *
from single_quadrotor import *

from scipy.integrate import solve_ivp, cumtrapz
from scipy.linalg import null_space, expm, inv, norm, eig

In [ ]:
# Fixed omega computation
try:
#     df = rosbag_pandas.bag_to_dataframe('/home/dcabecinhas/gazebo_logs/rigid_body_load_1_vehicle_2019-08-27-12-36-13.bag')
    df = rosbag_pandas.bag_to_dataframe('/home/dcabecinhas/gazebo_logs/rigid_body_load_1_vehicle_2019-08-28-12-47-19.bag')
except TypeError:
    print("Check if file exists!")

In [ ]:
# Filter only interesting trajectory segment
df = df.between_time('00:00:20', '00:00:50')

# Resample at 50Hz so we can do copmutations between dataframes
df = df.resample('20ms').last()
df = df.fillna(method='backfill')

List of columns that are not covariances

In [ ]:
# # List topics
# topics = [col for col in df.columns if 'cov' not in col]
# topics.sort()
# [print(elem) for elem in topics];

In [ ]:
def filter_topics(df,topics):
    df_filtered = df[[*topics]]
#     df_filtered = df_filtered.fillna(method='backfill')
#     df_filtered = df_filtered[df_filtered[df_filtered.columns[0]].notnull()]
    return df_filtered

In [ ]:
# Quadrotor - Gazebo

In [ ]:
topics = ['quad_odom__pose_pose_position_x',
          'quad_odom__pose_pose_position_y',
          'quad_odom__pose_pose_position_z']
df_pQ_gazebo = filter_topics(df, topics)

topics = ['quad_odom__twist_twist_linear_x',
          'quad_odom__twist_twist_linear_y',
          'quad_odom__twist_twist_linear_z']
df_vQ_gazebo = filter_topics(df, topics)

topics = ['quad_odom__pose_pose_position_x',
          'quad_odom__pose_pose_position_y',
          'quad_odom__pose_pose_position_z']
df_fQ_gazebo = filter_topics(df, topics)

topics = ['quad_odom__pose_pose_orientation_x',
            'quad_odom__pose_pose_orientation_y',
            'quad_odom__pose_pose_orientation_z',
            'quad_odom__pose_pose_orientation_w']
df_qQ_gazebo = filter_topics(df, topics)

topics = ['quad_odom__twist_twist_angular_x',
            'quad_odom__twist_twist_angular_y',
            'quad_odom__twist_twist_angular_z']
df_oQ_gazebo = filter_topics(df, topics)

topics = ['quad_force__wrench_torque_x',
            'quad_force__wrench_torque_y',
            'quad_force__wrench_torque_z']
df_tauQ_gazebo = filter_topics(df, topics)

In [ ]:
# Load - Gazebo

In [ ]:
topics = ['rb_link_odom__pose_pose_position_x',
          'rb_link_odom__pose_pose_position_y',
          'rb_link_odom__pose_pose_position_z']
df_pL_gazebo = filter_topics(df, topics)

topics = ['rb_link_odom__twist_twist_linear_x',
          'rb_link_odom__twist_twist_linear_y',
          'rb_link_odom__twist_twist_linear_z']
df_vL_gazebo = filter_topics(df, topics)

topics = ['rb_link_force__wrench_force_x',
            'rb_link_force__wrench_force_y',
            'rb_link_force__wrench_force_z']
df_fL_gazebo = filter_topics(df, topics)

topics = ['rb_link_odom__pose_pose_orientation_x',
            'rb_link_odom__pose_pose_orientation_y',
            'rb_link_odom__pose_pose_orientation_z',
            'rb_link_odom__pose_pose_orientation_w']
df_qL_gazebo = filter_topics(df, topics)

topics = ['rb_link_odom__twist_twist_angular_x',
            'rb_link_odom__twist_twist_angular_y',
            'rb_link_odom__twist_twist_angular_z']
df_oL_gazebo = filter_topics(df, topics)

topics = ['rb_link_force__wrench_torque_x',
            'rb_link_force__wrench_torque_y',
            'rb_link_force__wrench_torque_z']
df_tauL_gazebo = filter_topics(df, topics)

In [ ]:
# Quad - Mavros Odom

In [ ]:
topics = ['mavros_local_position_odom__pose_pose_position_x',
          'mavros_local_position_odom__pose_pose_position_y',
          'mavros_local_position_odom__pose_pose_position_z']
df_pQ_odom = filter_topics(df, topics)

topics = ['mavros_local_position_odom__twist_twist_linear_x',
          'mavros_local_position_odom__twist_twist_linear_y',
          'mavros_local_position_odom__twist_twist_linear_z']
df_vQ_odom = filter_topics(df, topics)

topics = ['mavros_local_position_odom__pose_pose_orientation_x',
            'mavros_local_position_odom__pose_pose_orientation_y',
            'mavros_local_position_odom__pose_pose_orientation_z',
            'mavros_local_position_odom__pose_pose_orientation_w']
df_qQ_odom = filter_topics(df, topics)

topics = ['mavros_local_position_odom__twist_twist_angular_x',
            'mavros_local_position_odom__twist_twist_angular_y',
            'mavros_local_position_odom__twist_twist_angular_z']
df_oQ_odom = filter_topics(df, topics)

In [ ]:
# Quad - Mavros Pose

In [ ]:
# topics = ['mavros_local_position_pose__pose_pose_position_x',
#           'mavros_local_position_pose__pose_pose_position_y',
#           'mavros_local_position_pose__pose_pose_position_z']
# df_pQ_pose = filter_topics(df, topics)

# topics = ['mavros_local_position_pose__pose_pose_orientation_x',
#             'mavros_local_position_pose__pose_pose_orientation_y',
#             'mavros_local_position_pose__pose_pose_orientation_z',
#             'mavros_local_position_pose__pose_pose_orientation_w']
# df_qQ_pose = filter_topics(df, topics)

In [ ]:
# Quad - Mavros IMU

In [ ]:
topics = ['mavros_imu_data__orientation_x',
            'mavros_imu_data__orientation_y',
            'mavros_imu_data__orientation_z',
            'mavros_imu_data__orientation_w']
df_qQ_imu = filter_topics(df, topics)

topics = ['mavros_imu_data__angular_velocity_x',
            'mavros_imu_data__angular_velocity_y',
            'mavros_imu_data__angular_velocity_z']
df_oQ_imu = filter_topics(df, topics)

In [ ]:
# Quad - Mavros Setpoint Position

In [ ]:
# topics = ['mavros_setpoint_position_local__pose_pose_position_x',
#           'mavros_setpoint_position_local__pose_pose_position_y',
#           'mavros_setpoint_position_local__pose_pose_position_z']
# df_pQ_setpoint = filter_topics(df, topics)

# topics = ['mavros_setpoint_position_local__pose_pose_orientation_x',
#             'mavros_setpoint_position_local__pose_pose_orientation_y',
#             'mavros_setpoint_position_local__pose_pose_orientation_z',
#             'mavros_setpoint_position_local__pose_pose_orientation_w']
# df_qQ_setpoint = filter_topics(df, topics)

In [ ]:
# Quad - Mavros Setpoint Target Position

In [ ]:
# topics = ['mavros_setpoint_raw_target_local__pose_pose_position_x',
#           'mavros_setpoint_raw_target_local__pose_pose_position_y',
#           'mavros_setpoint_raw_target_local__pose_pose_position_z']
# df_pQ_setpoint_target = filter_topics(df, topics)

# topics = ['mavros_setpoint_raw_target_local__pose_pose_orientation_x',
#             'mavros_setpoint_raw_target_local__pose_pose_orientation_y',
#             'mavros_setpoint_raw_target_local__pose_pose_orientation_z',
#             'mavros_setpoint_raw_target_local__pose_pose_orientation_w']
# df_qQ_setpoint_target = filter_topics(df, topics)

In [ ]:
# Quad - Mavros Setpoint Attitude

In [ ]:
topics = ['mavros_setpoint_raw_attitude__body_rate_x',
          'mavros_setpoint_raw_attitude__body_rate_y',
          'mavros_setpoint_raw_attitude__body_rate_z']
df_oQ_setpoint = filter_topics(df, topics)

topics = ['mavros_setpoint_raw_attitude__orientation_x',
            'mavros_setpoint_raw_attitude__orientation_y',
            'mavros_setpoint_raw_attitude__orientation_z',
            'mavros_setpoint_raw_attitude__orientation_w']
df_qQ_setpoint = filter_topics(df, topics)

topics = ['mavros_setpoint_raw_attitude__thrust']
df_TQ_setpoint = filter_topics(df, topics)

In [ ]:
# Quad - Mavros Setpoint Target Attitude

In [ ]:
topics = ['mavros_setpoint_raw_target_attitude__body_rate_x',
          'mavros_setpoint_raw_target_attitude__body_rate_y',
          'mavros_setpoint_raw_target_attitude__body_rate_z']
df_oQ_setpoint_target = filter_topics(df, topics)

topics = ['mavros_setpoint_raw_target_attitude__orientation_x',
            'mavros_setpoint_raw_target_attitude__orientation_y',
            'mavros_setpoint_raw_target_attitude__orientation_z',
            'mavros_setpoint_raw_target_attitude__orientation_w']
df_qQ_setpoint_target = filter_topics(df, topics)

topics = ['mavros_setpoint_raw_target_attitude__thrust']
df_TQ_setpoint_target = filter_topics(df, topics)

In [ ]:
# Quad - Mavros Actuation Target 

In [ ]:
topics = ['mavros_target_actuator_control__controls0',
          'mavros_target_actuator_control__controls1',
          'mavros_target_actuator_control__controls2',]
df_tauQ_target = filter_topics(df, topics)

topics = ['mavros_target_actuator_control__controls3',]
df_TQ_target = filter_topics(df, topics)

In [ ]:
# fig, axes = subplots(4, 3, figsize=(20, 15), sharex=True)
fig, axes = subplots(4, 3, figsize=(20, 15))

plot_style = {
#     'marker':'-',
    'markersize':0.2,
    'grid':True,
    'legend':False, 
             }

df_pQ_gazebo.plot(ax=axes[0,0], **plot_style)
df_vQ_gazebo.plot(ax=axes[0,1], **plot_style)
df_fQ_gazebo.plot(ax=axes[0,2], **plot_style)

df_pL_gazebo.plot(ax=axes[1,0], **plot_style)
df_vL_gazebo.plot(ax=axes[1,1], **plot_style)
df_fL_gazebo.plot(ax=axes[1,2], **plot_style)

df_qQ_gazebo.plot(ax=axes[2,0], **plot_style)
df_oQ_gazebo.plot(ax=axes[2,1], **plot_style)
df_tauQ_gazebo.plot(ax=axes[2,2], **plot_style)

df_qL_gazebo.plot(ax=axes[3,0], **plot_style)
df_oL_gazebo.plot(ax=axes[3,1], **plot_style)
df_tauL_gazebo.plot(ax=axes[3,2], **plot_style)

# axes[3,2].set_xlim(pd.Timestamp('1970-01-01 00:00:15'), pd.Timestamp('1970-01-01 00:01:15'));

In [ ]:
topics = ['mavros_setpoint_raw_target_local__position_x',
          'mavros_setpoint_raw_target_local__position_y',
          'mavros_setpoint_raw_target_local__position_z']
mav_target_position = filter_topics(df, topics)

topics = ['mavros_setpoint_raw_target_local__yaw']
mav_target_yaw = filter_topics(df, topics)

topics = ['mavros_setpoint_raw_target_attitude__orientation_x',
          'mavros_setpoint_raw_target_attitude__orientation_y',
          'mavros_setpoint_raw_target_attitude__orientation_z',
          'mavros_setpoint_raw_target_attitude__orientation_w']
mav_target_attitude = filter_topics(df, topics)

topics = ['mavros_target_actuator_control__controls0',
          'mavros_target_actuator_control__controls1',
          'mavros_target_actuator_control__controls2',
          'mavros_target_actuator_control__controls3']
mav_actuators = filter_topics(df, topics)

In [ ]:
fig, axes = subplots(2, 3, figsize=(20, 8))

plot_style = {
    'linestyle':'',
    'marker':'.',
    'markersize':1,
    'grid':True,
    'legend':False, 
             }

df_pQ_gazebo.plot(ax=axes[0,0], **plot_style)
# mav_target_position.plot(ax=axes[0,0], **plot_style)

df_pL_gazebo.plot(ax=axes[0,1], **plot_style)
df_pQ_gazebo.plot(ax=axes[0,1], **plot_style)

(-mav_target_attitude).plot(ax=axes[1,0], **plot_style)
df_qQ_gazebo.plot(ax=axes[1,0], **plot_style)

df_oL_gazebo.plot(ax=axes[0,2], **plot_style)
mav_actuators.iloc[:,0:3].plot(ax=axes[1,1], **plot_style)

df_tauL_gazebo.plot(ax=axes[1,2], **plot_style)
# mav_actuators.plot(ax=axes[1,1], **plot_style)

In [ ]:
fig, axes = subplots(1, 1, figsize=(10, 6))

plot(df_pL_gazebo.index, df_pL_gazebo)
plot(df_pQ_gazebo.index, df_pQ_gazebo)
grid(True)

In [ ]:
fig, axes = subplots(1, 1, figsize=(10, 6))

plot_style = {
    'linestyle':'',
    'marker':'.',
    'markersize':2,
#     'grid':True,
#     'legend':False, 
             }

plot(df_qQ_gazebo.index, df_qQ_gazebo.iloc[:,0:2], **plot_style)
plot(mav_target_attitude.index, mav_target_attitude.iloc[:,0:2], **plot_style)
grid(True)

In [ ]:
fig, axes = subplots(1, 1, figsize=(10,6))

plot(df_qQ_gazebo.index, df_qQ_gazebo.iloc[:,0:2])
plot(mav_target_attitude.index, -mav_target_attitude.iloc[:,0:2])
grid(True)

In [ ]:
fig, axes = subplots(1, 1, figsize=(10,6))

plot(df_qQ_gazebo.index, df_qQ_gazebo.iloc[:,0:2])
plot(mav_target_attitude.index, -mav_target_attitude.iloc[:,0:2])
# plot(df_oL_gazebo.index, df_oL_gazebo)
grid(True)

In [ ]:
fig, axes = subplots(1, 1, figsize=(10,6))

# plot(df_qL_gazebo.index, df_qL_gazebo.iloc[:,0:2])
# plot(mav_target_attitude.index, -mav_target_attitude.iloc[:,0:2])
plot(df_oL_gazebo.index, df_oL_gazebo, linestyle="", marker='.', markersize=1)
grid(True)

In [ ]:
fig, axes = subplots(1, 1, figsize=(10,6))

# plot(df_qL_gazebo.index, df_qL_gazebo.iloc[:,0:2])
# plot(mav_target_attitude.index, -mav_target_attitude.iloc[:,0:2])
plot(df_tauL_gazebo.index, df_tauL_gazebo, linestyle="", marker='.', markersize=1)
grid(True)

In [ ]:
t = pd.to_timedelta(df_pQ_gazebo.index - df_pQ_gazebo.index[0] ).total_seconds().to_numpy()

pQ = transform_position_I(df_pQ_gazebo.values.reshape(-1,1,3,1))
pL = transform_position_I(df_pL_gazebo.values.reshape(-1,1,3,1))
vQ = transform_position_I(df_vQ_gazebo.values.reshape(-1,1,3,1))
vL = transform_position_I(df_vL_gazebo.values.reshape(-1,1,3,1))
aQ = transform_position_I(df_fQ_gazebo.values.reshape(-1,1,3,1))
aL = transform_position_I(df_fL_gazebo.values.reshape(-1,1,3,1))

pQ = np.nan_to_num(pQ)
pL = np.nan_to_num(pL)
vQ = np.nan_to_num(vQ)
vL = np.nan_to_num(vL)
aQ = np.nan_to_num(aQ)
aQ = np.nan_to_num(aL)

actuation = mav_actuators.values.reshape(-1,1,4,1)
target_attitude = mav_target_attitude.values.reshape(-1,1,4,1)

In [ ]:
def skew(v):
    # New shape is v shape with last dimension 3 instead of 1 (3x1 vector --> 3x3 matrix)
    s = np.asarray(v.shape)
    s[-1] = 3
    
    # Vectorize
    # Maintains the shape of v for vi
    v0 = v[...,0,0,np.newaxis,np.newaxis]
    v1 = v[...,1,0,np.newaxis,np.newaxis]
    v2 = v[...,2,0,np.newaxis,np.newaxis]

    z = np.zeros_like(v0)
    
    # TODO: Check that dim(v) = 3
    res = np.concatenate([z, -v2, v1,
                    v2, z, -v0,
                    -v1, v0, z],axis=-2)
    
    return res.reshape(s)

def unskew(M):
    # New shape is M shape with last dimension 1 instead of 3 (3x3 matrix --> 3x1 vector)
    s = np.asarray(M.shape)
    s[-1] = 1

    # TODO: Check if M is anti-symmetric
#     TODO: stack along axis=-1 instead of using reshape()
    
    res = np.stack([-M[...,1,2].reshape(-1,1),
                  M[...,0,2].reshape(-1,1),
                  -M[...,0,1].reshape(-1,1)],axis=1).reshape(s)
    
    return res

# TODO: Check that unskew(skew(v)) == v
# TODO: Do not make assumptions on shape of input

In [ ]:
def mt(M):
# Matrix transpose no matter what is M shape.  M = (...) x n x m or M = (...) x m x n
    
    return np.swapaxes(M,-1,-2)

In [ ]:
l = norm(pL - pQ, axis=-2, keepdims=True)

q = (pL - pQ) / norm(pL - pQ, axis=-2, keepdims=True)
q = np.nan_to_num(q)

dq = (np.eye(3) - q @ mt(q)) @ (vL - vQ) / norm(pL - pQ, axis=-2, keepdims=True)
dq = np.nan_to_num(dq)

o = - skew(q) @ dq

n = len(pQ)

In [ ]:
plot(np.squeeze(l[:-1]))

In [ ]:
RL = np.repeat(np.eye(3).reshape(-1,1,3,3),n,axis=0)
oL = np.zeros([n,1,3,1])
RQ = np.repeat(np.eye(3).reshape(-1,1,3,3),n,axis=0)
oQ = np.zeros([n,1,3,1])
delta_TLd = np.zeros([n,1,3,1])
V = np.zeros([n,1,1,1])

# print("===================================")

y = pack_solution(pL,
            vL,
            RL,
            oL,
            delta_TLd,
            q,
            o,
            RQ,
            oQ,
            V)

t = pd.to_timedelta(df_pQ_gazebo.index - df_pQ_gazebo.index[0] ).total_seconds().to_numpy()

dy, state = process_state(t,mt(y))

In [ ]:
fig, axes = subplots(5, 3, figsize=(20, 16))

all_axes = [ax.grid() for sublist in axes for ax in sublist]

axes[0,0].plot(t,squeeze(pL))
axes[0,1].plot(t,squeeze(vL))
axes[0,2].plot(t,squeeze(aL))

axes[1,0].plot(t,squeeze(pQ))
axes[1,1].plot(t,squeeze(vQ))
axes[1,2].plot(t,squeeze(aQ))

axes[2,0].plot(t,squeeze(q))
axes[2,1].plot(t[:-1],squeeze(o[:-1,...]))
# axes[2,1].set_ylim([-0.01,0.01])
axes[2,2].plot(t,squeeze(state.u_perp))

axes[3,0].plot(t,squeeze(state.q-state.qd))
axes[3,1].plot(t,squeeze(state.F-state.Fd))
# axes[3,2].plot(t,squeeze(state.F))
axes[3,2].plot(t,squeeze(state.qF))

axes[4,0].plot(t,squeeze(mav_target_attitude))
axes[4,1].plot(t,squeeze(mav_actuators))
# axes[3,2].plot(t,squeeze(state.F))
# axes[4,2].plot(t,squeeze(state.qF))

# target_attitude

# Comparisons

## Position

In [ ]:
fig, axes = subplots(1, 3, figsize=(15, 4))

all_axes = [ax.grid() for ax in axes]

axes[0].plot(t,squeeze(df_pQ_gazebo.values))
axes[1].plot(t,squeeze(df_pQ_odom))
axes[2].plot(t,squeeze(df_pQ_gazebo.values - df_pQ_odom))

In [ ]:
fig, axes = subplots(2, 3, figsize=(15, 4))

all_axes = [ax.grid() for sublist in axes for ax in sublist]

axes[0,0].plot(t,squeeze(df_pQ_gazebo.values))
axes[0,1].plot(t,squeeze(df_vQ_gazebo.values))
# axes[2].plot(t,squeeze(cumtrapz(df_vQ_gazebo.values,t,axis=0,initial=0)))
axes[0,2].plot(t,squeeze(df_pQ_gazebo.values-cumtrapz(df_vQ_gazebo.values,t,axis=0,initial=0)-df_pQ_gazebo.values[0,:]))
axes[1,0].plot(t,squeeze(df_pL_gazebo.values))
axes[1,1].plot(t,squeeze(df_vL_gazebo.values))
# axes[2].plot(t,squeeze(cumtrapz(df_vQ_gazebo.values,t,axis=0,initial=0)))
axes[1,2].plot(t,squeeze(df_pL_gazebo.values-cumtrapz(df_vL_gazebo.values,t,axis=0,initial=0)-df_pL_gazebo.values[0,:]))

## Attitude

In [ ]:
df_qQ_gazebo.values[:,0] = -df_qQ_gazebo.values[:,0] * np.sign(df_qQ_gazebo.values[:,3])
df_qQ_gazebo.values[:,1] = -df_qQ_gazebo.values[:,1] * np.sign(df_qQ_gazebo.values[:,3])
df_qQ_gazebo.values[:,2] = -df_qQ_gazebo.values[:,2] * np.sign(df_qQ_gazebo.values[:,3])
df_qQ_gazebo.values[:,3] = -df_qQ_gazebo.values[:,3] * np.sign(df_qQ_gazebo.values[:,3])

In [ ]:
fig, axes = subplots(3, 3, figsize=(20, 15))

all_axes = [ax.grid() for sublist in axes for ax in sublist]

plot_options = {'linestyle':"", 'marker':'.', 'markersize':1}

axes[0,0].plot(t,squeeze(df_qQ_gazebo.values), **plot_options)
axes[0,1].plot(t,squeeze(df_qQ_imu.values), **plot_options)
axes[0,2].plot(t,squeeze(df_qQ_odom.values), **plot_options)
axes[1,0].plot(t,squeeze(df_qQ_setpoint.values), **plot_options)
axes[1,1].plot(t,squeeze(df_qQ_setpoint_target.values), **plot_options)
axes[1,2].plot(t,squeeze(df_qQ_imu.values - df_qQ_odom.values), **plot_options)
axes[2,0].plot(t,squeeze(df_qQ_gazebo.values - df_qQ_imu.values), **plot_options)
axes[2,1].plot(t,squeeze(df_qQ_gazebo.values - df_qQ_odom.values), **plot_options)
axes[2,2].plot(t,squeeze(df_qQ_gazebo.values - df_qQ_setpoint.values), **plot_options)

## Load angle

In [ ]:
fig, axes = subplots(3, 3, figsize=(20, 15))

all_axes = [ax.grid() for sublist in axes for ax in sublist]

plot_options = {'linestyle':"", 'marker':'.', 'markersize':1}

axes[0,0].plot(t,squeeze(state.q), **plot_options)
axes[0,1].plot(t,squeeze(state.qd), **plot_options)
axes[0,2].plot(t,squeeze(state.q - state.qd), **plot_options)
axes[1,0].plot(t,squeeze(state.q_actuation), **plot_options)
axes[1,1].plot(t,squeeze(state.e_q), **plot_options)
axes[1,2].plot(t,squeeze(state.e_oq), **plot_options)

## State vars

In [ ]:
fig, axes = subplots(3, 3, figsize=(20, 15))

all_axes = [ax.grid() for sublist in axes for ax in sublist]

plot_options = {'linestyle':"", 'marker':'.', 'markersize':1}

axes[0,0].plot(t,squeeze(state.mud), **plot_options)
axes[0,1].plot(t,squeeze(state.mu), **plot_options)
axes[0,2].plot(t,squeeze(state.mu - state.mud), **plot_options)

axes[1,0].plot(t,squeeze(state.Fd), **plot_options)
axes[1,1].plot(t,squeeze(state.TLd), **plot_options)

axes[2,0].plot(t,squeeze(state.u), **plot_options)
axes[2,1].plot(t,squeeze(state.u_parallel), t,squeeze(state.mu), **plot_options)
axes[2,2].plot(t,squeeze(state.u_perp), **plot_options)

In [ ]:
# u_parallel = mu + mQ * lq * norm(oq, axis=-2, keepdims=True)**2 * q + mQ * q @ mt(q) @ a
fig, axes = subplots(3, 3, figsize=(20, 15))

all_axes = [ax.grid() for sublist in axes for ax in sublist]

plot_options = {'linestyle':"", 'marker':'.', 'markersize':1}

axes[0,0].plot(t,squeeze(state.mu), **plot_options)
axes[0,1].plot(t,squeeze(mQ * lq * norm(state.oq, axis=-2, keepdims=True)**2 * state.q), **plot_options)
axes[0,2].plot(t,squeeze(mQ * state.q @ mt(state.q) @ state.a), **plot_options)
axes[1,0].plot(t,squeeze(state.q))
axes[1,1].plot(t,squeeze(state.dq))
axes[1,2].plot(t,squeeze(state.oq))
axes[2,0].plot(t,squeeze(q))
axes[2,1].plot(t,squeeze((np.eye(3) - q @ mt(q)) @ (vL - vQ) / norm(pL - pQ, axis=-2, keepdims=True)))
axes[2,2].plot(t,squeeze(q),t,squeeze((vL - vQ) / norm(pL - pQ, axis=-2, keepdims=True)))

In [ ]:
fig, axes = subplots(5, 3, figsize=(20, 15))

all_axes = [ax.grid() for sublist in axes for ax in sublist]
# all_axes = [ax.set_xlim([20,30]) for sublist in axes for ax in sublist]
plot_options = {'linestyle':"", 'marker':'.', 'markersize':1}

axes[0,0].plot(t,squeeze(state.q), **plot_options)
axes[0,1].plot(t,squeeze(state.oq), **plot_options)
axes[0,1].set_ylim([-1,1])
axes[0,2].plot(t,squeeze(state.doq), **plot_options)

axes[1,0].plot(t,squeeze(state.qd),t,squeeze(state.qd), **plot_options)
axes[1,1].plot(t,squeeze(state.oqd), **plot_options)
axes[1,1].set_ylim([-1,1])
axes[1,2].plot(t,squeeze(state.doqd), **plot_options)

axes[2,0].plot(t,squeeze(state.q),t,squeeze(state.qd), **plot_options)
axes[2,1].plot(t,squeeze(state.oq),t,squeeze(state.oqd), **plot_options)
axes[2,1].set_ylim([-1,1])
axes[2,2].plot(t,squeeze(state.zoq), **plot_options)

axes[3,0].plot(t,squeeze(state.oq), **plot_options)
axes[3,1].plot(t[:-1],50*diff(squeeze(state.oq),axis=0), **plot_options)
axes[3,1].set_ylim([-2,2])
axes[3,2].plot(t,squeeze(state.doq), **plot_options)

In [ ]:
# u_perp_d =  (mQ * lq) * skew(q) @ (
#         - 1 / lq * skew(q) @ a 
#         - skew(dq) @ skew(q) @ oqd
#         - skew(q) @ skew(dq) @ oqd
#         - skew(q) @ skew(q) @ doqd
#         - 1 / L_q * (
#                 + (mt(dqd) @ TLd) * skew(q)
#                 + (mt(qd) @ dTLd) * skew(q)
#                 + (mt(qd) @ TLd) * skew(dq)
#             ) @ (
#                     1/mL * (x_pv * pe + ve)
#                 )
#         - 1 / L_q * (mt(qd) @ TLd) * skew(q) @ (
#             1/mL * (x_pv * ve + ae)
#         ) 
#         + koq / L_oq * zoq
#         + L_q / L_oq * skew(q) @ qd
#     )

In [ ]:
fig, axes = subplots(2, 2, figsize=(20, 15))

all_axes = [ax.set_xlim([10,20]) for sublist in axes for ax in sublist]
all_axes = [ax.grid() for sublist in axes for ax in sublist]

axes[0,0].plot(t[:-1],50*diff(squeeze(state.oq),axis=0)[:,0], **plot_options)
axes[0,0].set_ylim([-2,2])
axes[0,0].plot(t,squeeze(state.doq)[:,0], **plot_options)

axes[0,1].plot(t[:-1],50*diff(squeeze(state.oq),axis=0)[:,1], **plot_options)
axes[0,1].set_ylim([-2,2])
axes[0,1].plot(t,squeeze(state.doq)[:,1], **plot_options)

axes[1,0].plot(t[:-1],50*diff(squeeze(state.oq),axis=0)[:,2], **plot_options)
axes[1,0].set_ylim([-2,2])
axes[1,0].plot(t,squeeze(state.doq)[:,2], **plot_options)

axes[1,1].plot(t[:-1],50*diff(squeeze(state.oq),axis=0), **plot_options)
axes[1,1].set_ylim([-2,2])
axes[1,1].plot(t,squeeze(state.doq), **plot_options)

In [ ]:
fig, axes = subplots(2, 2, figsize=(20, 15))

all_axes = [ax.set_xlim([10,20]) for sublist in axes for ax in sublist]
all_axes = [ax.grid() for sublist in axes for ax in sublist]

axes[0,0].plot(t[:-1],50*diff(squeeze(state.oq),axis=0)[:,0], **plot_options)
axes[0,0].set_ylim([-2,2])
axes[0,0].plot(t,squeeze(- 1 / (mQ * lq) * skew(state.q) @ state.u_perp)[:,0], **plot_options)

axes[0,1].plot(t[:-1],50*diff(squeeze(state.oq),axis=0)[:,1], **plot_options)
axes[0,1].set_ylim([-2,2])
axes[0,1].plot(t,squeeze(- 1 / (mQ * lq) * skew(state.q) @ state.u_perp)[:,1], **plot_options)

axes[1,0].plot(t[:-1],50*diff(squeeze(state.oq),axis=0)[:,2], **plot_options)
axes[1,0].set_ylim([-2,2])
axes[1,0].plot(t,squeeze(- 1 / (mQ * lq) * skew(state.q) @ state.u_perp)[:,2], **plot_options)

axes[1,1].plot(t[:-1],50*diff(squeeze(state.oq),axis=0), **plot_options)
axes[1,1].set_ylim([-2,2])
axes[1,1].plot(t,squeeze(- 1 / (mQ * lq) * skew(state.q) @ state.u_perp), **plot_options)

In [ ]:
fig, axes = subplots(4, 3, figsize=(20, 15))

all_axes = [ax.grid() for sublist in axes for ax in sublist]
all_axes = [ax.set_xlim([10,20]) for sublist in axes for ax in sublist]
plot_options = {'linestyle':"", 'marker':'.', 'markersize':1}

axes[0,0].plot(t,squeeze(state.p), **plot_options)
axes[0,1].plot(t,squeeze(state.v), **plot_options)
axes[0,2].plot(t,squeeze(state.dv), **plot_options)
# axes[0,2].plot(t,squeeze(state.doq), **plot_options)

axes[1,0].plot(t,squeeze(df_pL_gazebo.values))
axes[1,1].plot(t,squeeze(df_vL_gazebo.values))
axes[1,2].plot(t[:-1],50*diff(squeeze(df_vL_gazebo.values),axis=0))

axes[2,0].plot(t,squeeze(state.p), t,squeeze(df_pL_gazebo.values))
axes[2,1].plot(t,squeeze(state.v), t,squeeze(df_vL_gazebo.values))
axes[2,2].plot(t,squeeze(state.dv), t[:-1],50*diff(squeeze(df_vL_gazebo.values),axis=0))

axes[3,0].plot(t,squeeze(state.dv), **plot_options)
axes[3,1].plot(t,squeeze(state.Fd), **plot_options)
axes[3,2].plot(t,squeeze(state.F), **plot_options)
# axes[0,2].plot(t,squeeze(state.doq), **plot_options)


In [ ]:
fig, axes = subplots(3, 3, figsize=(20, 15))

all_axes = [ax.grid() for sublist in axes for ax in sublist]
all_axes = [ax.set_xlim([10,20]) for sublist in axes for ax in sublist]
plot_options = {'linestyle':"", 'marker':'.', 'markersize':1}

axes[0,0].plot(t,squeeze(df_qQ_gazebo.values), **plot_options)
axes[0,1].plot(t,squeeze(df_qQ_setpoint_target.values), **plot_options)
axes[0,2].plot(t,squeeze(df_qQ_gazebo.values), t,squeeze(df_qQ_setpoint_target.values), **plot_options)


axes[1,0].plot(t,squeeze(df_TQ_target), **plot_options)
axes[1,1].plot(t,squeeze(df_TQ_setpoint), **plot_options)
axes[1,2].plot(t,squeeze(df_TQ_target),t,squeeze(df_TQ_setpoint), **plot_options)

In [ ]:
fig, axes = subplots(2, 2, figsize=(20, 15))

all_axes = [ax.set_xlim([10,20]) for sublist in axes for ax in sublist]
all_axes = [ax.grid() for sublist in axes for ax in sublist]

axes[0,0].plot(t[:-1],50*diff(squeeze(state.oq),axis=0)[:,0], **plot_options)
axes[0,0].set_ylim([-2,2])
axes[0,0].plot(t,squeeze( 1 / (mQ * lq) * skew(state.q) @ state.u_perp)[:,0], **plot_options)

axes[0,1].plot(t[:-1],50*diff(squeeze(state.oq),axis=0)[:,1], **plot_options)
axes[0,1].set_ylim([-2,2])
axes[0,1].plot(t,squeeze( 1 / (mQ * lq) * skew(state.q) @ state.u_perp)[:,1], **plot_options)

axes[1,0].plot(t[:-1],50*diff(squeeze(state.oq),axis=0)[:,2], **plot_options)
axes[1,0].set_ylim([-2,2])
axes[1,0].plot(t,squeeze( 1 / (mQ * lq) * skew(state.q) @ state.u_perp)[:,2], **plot_options)

axes[1,1].plot(t[:-1],50*diff(squeeze(state.oq),axis=0), **plot_options)
axes[1,1].set_ylim([-2,2])
axes[1,1].plot(t,squeeze( 1 / (mQ * lq) * skew(state.q) @ state.u_perp), **plot_options)